In [17]:
import collections
import pandas as pd
import datetime
import numpy as np
import json

In [18]:
data_fr = pd.read_csv('../data/df_fr.csv', sep=',')
corres_ICD10 = pd.read_csv('../data/corres_ICD10.csv', sep=';',usecols=[0,1,2,4], header=None)

In [19]:
data_fr.columns = ['categorie','unit','sex','age','icd10','geo','2015','2014','2013','2012','2011','2010','2009','2008','2007','2006','2005','2004','2003','2002','2001','2000','1999','1998','1997','1996','1995','1994']

In [20]:
categorie_0 = ['A-R_V-Y']
categorie_1 = ['A-U','V01-Y89']
categorie_2 = ['A_B','C00-D48','E','F','TOXICO','G_H','I','J','K','L','M,','N','O','P','Q','R']

In [21]:
corres_list = corres_ICD10[0].values.tolist()

In [22]:
categorie_3 = corres_list
for elt in categorie_0:
    try:
        categorie_3.remove(elt);
    except:
        pass
for elt in categorie_1:
    try:
        categorie_3.remove(elt);
    except:
        pass
for elt in categorie_2:
    try:
        categorie_3.remove(elt);
    except:
        pass

In [23]:
def categorize(ele):
    if ele in categorie_3:
        return 3
    else:
        if ele in categorie_1:
            return 1
        else:
            if ele in categorie_2:
                return 2
            else:
                return 0
data_fr['categorie'] = data_fr['icd10'].apply(categorize)

In [24]:
data_fr.to_csv('../data/data_fr_new.csv')

In [11]:
icd10_dict = corres_ICD10.set_index(0).loc[:,1].to_dict()

## Partie SEB

In [26]:
for col in data_fr.columns[6:]:
    data_fr[col] = data_fr[col].str.strip().replace({":":"NaN", ": z":"NaN"}).astype('float64')

In [27]:
data_fr.insert(loc=4, value=data_fr.icd10.replace(icd10_dict).values, column="maladies")

NameError: name 'icd10_dict' is not defined

In [14]:
data_fr.drop(columns=["unit"], inplace=True)
data_fr.head(2)

,categorie,sex,age,maladies,icd10,geo,2015,2014,2013,2012,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994
0,0,F,TOTAL,Toutes causes de mortalit� (A00-Y89) sauf S00-T98,A-R_V-Y,FR,864.04,810.56,816.66,824.57,...,868.2,839.1,835.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,F,TOTAL,Toutes causes de mortalit� (A00-Y89) sauf S00-T98,A-R_V-Y,FR10,602.05,569.34,585.50,593.06,...,684.3,640.7,649.4,649.2,660.7,670.5,671.1,681.2,686.3,669.9


In [15]:
data_fr.to_csv('data_fr_new.csv')

In [51]:
df = pd.pivot_table(data_fr, values=data_fr.columns[6:], index=['geo', 'icd10'], aggfunc=np.sum)

In [17]:
# colIndex=pd.MultiIndex.from_product([data_fr.geo.unique(),
#                                     data_fr.icd10.unique()],
#                                names=['Region','Code'])

In [20]:
# df = data_fr.set_index(['geo', 'sex', 'age', 'icd10']).drop(columns=['categorie', 'unit'])
df = data_fr.groupby(['geo', 'sex', 'age', 'icd10','maladies']).sum()

df.to_json("test.json")

def nested_dict():
    return collections.defaultdict(nested_dict)
result = nested_dict()

In [21]:
for row in df.itertuples(name="parent"):
    break
    result[row.Index[0]][row.Index[1]]['2015'] = row._1

In [22]:
out = json.dumps(result)
# out = data_fr.groupby(['geo', 'sex', 'age', 'icd10']).sum()["2015"].to_json(orient='table')

In [23]:
with open('data.json', 'w') as f:
    f.write(out)